# Dynamic Time Warping

Dynamic time warping (dtw) is the comparison between two sequences. This generally the comparison of two data points of different time frames. This has been used in accelarations and decelrations, audio, video, graphics data, etc. There are specific rules of dtw specified below:

- Every index from the first sequence must be matched with one or more indices from the other sequence, and vice versa.
- The first index from the first sequence must be matched with the first index from the other sequence (but it does not have to be its only match)
- The last index from the first sequence must be matched with the last index from the other sequence (but it does not have to be its only match)
- The mapping of the indices from the first sequence to indices from the other sequence must be monotonically increasing, and vice versa, i.e. if $j>i$ are indices from the first sequence, then there must not be two indices $l>k$ in the other sequence, such that index $i$ is matched with index $l$ and index $j$ is matched with index $k$, and vice versa.

A good explanation is here: https://rtavenar.github.io/blog/dtw.html

The following path $\pi$ of length $K$ is a sequence of $K$ index pairs $(i_{0},j_{0}),...,(i_{K-1},j_{K-1})$.

$$DTW_q(\mathbf{x}, \mathbf{x}^\prime) =
    \min_{\pi \in \mathcal{A}(\mathbf{x}, \mathbf{x}^\prime)}
        \left( \sum_{(i, j) \in \pi} d(x_i, x^\prime_j)^q \right)^{\frac{1}{q}}$$

Where $\mathcal{A}(\mathbf{x}, \mathbf{x}^\prime)$ is the set of all admissible paths, i.e. the set of paths $\pi$ such that:
- $\pi$ is a sequence $[\pi_{0},...,\pi_{K-1}]$ of index pairs $\pi_{k} = (i_{k},j_{k})$ with $0\le i_{k}<n$ and $0\le j_{k} < m$
- $\pi = (0,0)$ and $\pi_{K-1} = (n-1,m-1)$
- for all $k > 0, \pi_{k} = (i_{k},j_{k})$ is related $\pi_{K-1} = (i_{K-1},j_{K-1})$ as follows:
    - $i_{k-1} \leq i_k \leq i_{k-1} + 1$
    - $j_{k-1} \leq j_k \leq j_{k-1} + 1$

We can obtain both the **distance** and the **mapping** of the points. We can use constraints known as **Sakaoe-Chiba** for a constant-width band of a radius based on window size and another constraint known as **Itakura** parallelogram to set the maximum sloe for the alignment paths.

We will keep the constraints global.

In [ ]:
def fast_dtw_label(data1, data2, label):

  data_label1 = data1[label]
  data_label2 = data2[label]

  distance = dtw.distance_fast(data_label1, data_label2)
  print("DISTANCE:", distance)
  distance, paths = dtw.warping_paths_fast(data_label1, data_label2)
  print("DISTANCE:", distance, "PATHS", paths)
  path = dtw.warping_path_fast(data_label1, data_label2)
  return path

With this method, we can obtain the ditance, which is the total distance between the points from the dtw, and the paths. Using the paths, we can transform the data to make it the same length.

In [ ]:
# iterating through the path_mapping to create two similar graphs
def transform_graphs_from_dtw_path(data1, data2, path_mapping, label):

  new_data1 = np.array([data1[label][map[0]] for map in path_mapping])
  new_data2 = np.array([data2[label][map[1]] for map in path_mapping])


  return new_data1, new_data2

We can obtain the `data_list` for t he listing of data. After, we obtrain the path listing.

In [ ]:
  data_list = [ad.load_data_sensors(sys.argv[x]) for x in range(1, len(sys.argv))]

  print("calculating dtw...")
  path = fast_dtw_label(data_list[0], data_list[1], "MeasurementsCurrent")
  print("Finished calculating dtw path :)")


We can then take every 200th value to visually compare the connection between two graphs.

In [ ]:
path_vis = path[::200]
  ad.compare_data(data_list[0]["MeasurementsCurrent"], data_list[1]["MeasurementsCurrent"], path_vis)

We can finally apply the transformation of the data to see the similarties.

In [ ]:
  new_data1, new_data2 = transform_graphs_from_dtw_path(data_list[0], data_list[1], path, "MeasurementsCurrent")
  print(type(new_data1), type(new_data2), len(new_data1), len(new_data2))
  ad.compare_data(new_data1, new_data2)

We can apply this warping to numerous algorithms that **require the dimension to be the same** such as **deep learning algorithms** and **correlation algorithms**. We can also use the `distance` to cluster the comparison betweeen the data points. The highest the distance, the more different the datasets are.

We can see the mappings and the transformation below:

![Image](..\images\dtw_mapping.png)
Mapping of two different melts
![Image](..\images\dtw_translation.png)
Transformation of two different melts